# Capstone Project from E. Wesner

This notebook will be mainly used for the capstone project of Coursera Data Science

In [1]:
import pandas as pd
import numpy as np
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Neighborhoods of Toronto "Dataimport and Cleaning"

**Import the Data from Wikipedia**

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup

In [3]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data = requests.get (URL)
soup = BeautifulSoup(html_data.content, 'html.parser')


In [6]:
#Empty dataframe with cols
neighborhoods_toronto = pd.DataFrame(columns = ["PostalCode","Borough","Neighborhood"])

#from the Soup pick the relevant data
table = soup.find_all("tbody")[0]

#extract the table and values
for row in table.find_all("tr"):
    col = row.find_all("td")
    textlist = [ele.text.strip()for ele in col]
    if textlist:                    #falls in text was drin steht, also nicht [] leer ist
        #print(text)
        for i in range (len(textlist)):
            einzeln = textlist[i]
            postcode = einzeln[0:3]
            schneiden = einzeln[3:].find("(")
            boro = einzeln[3:schneiden+3]   
            neigh = einzeln[schneiden+4:-1]
            neigh = neigh.replace(' / ', ' , ')
            neighborhoods_toronto = neighborhoods_toronto.append({"PostalCode":postcode,"Borough":boro,"Neighborhood":neigh}, 
                                                        ignore_index = True)
neighborhoods_toronto.shape

(180, 3)

In [7]:
neighborhoods_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,,Not assigne
1,M2A,,Not assigne
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"


In [8]:
#Not assignet delete
neighborhoods_toronto1=neighborhoods_toronto.replace('Not assigne', np.NaN)
neighborhoods_toronto2=neighborhoods_toronto1.replace('', np.NaN)
neighborhoods_toronto=neighborhoods_toronto2.dropna(subset=["Borough"],axis=0)
neighborhoods_toronto.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government


In [9]:
neighborhoods_toronto.shape

(103, 3)

In [10]:
#create the new index
neighborhoods_toronto.reset_index(drop=True,inplace=True)
neighborhoods_toronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [9]:
#searching for not assigned neighborhood but a borough
missing_data=neighborhoods_toronto.isnull()
for columns in missing_data.columns.values.tolist():
    print (columns)
    print (missing_data[columns].value_counts())
    print(" ")

PostalCode
False    103
Name: PostalCode, dtype: int64
 
Borough
False    103
Name: Borough, dtype: int64
 
Neighborhood
False    103
Name: Neighborhood, dtype: int64
 


No such data was found

In [11]:
print("The Shape of the DataFrame is: " + str(neighborhoods_toronto.shape))

The Shape of the DataFrame is: (103, 3)


# Part 2   Get the Latitude and Longitude

In [12]:
#Empty dataframe with cols
neighborhoods_coordinates = pd.DataFrame(columns = ["Postal Code","Latitude","Longitude"])
neighborhoods_coordinates.head()

,Postal Code,Latitude,Longitude


**Tried to use geocoder, but I didn't succeed**

In [ ]:
!pip install geocoder
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

for i in range (neighborhoods_toronto.shape[0]):
    postal_code = neighborhoods_toronto.at[neighborhoods_toronto.index[i],'Postal Code'] 
    print(postal_code)
    #loop until you get the coordinates
    while(lat_lng_coords is None):        
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    neighborhoods_coordinates = neighborhoods_coordinates.append({"Postal Code":postal_code,"Latitude":latitude,"Longitude":longitude}, 
                                                        ignore_index = True)
neighborhoods_coordinates.shape

**This code was created from the Notebook by importing the CSV-file Geospatial_Coordinates.csv**

In [13]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_0041bdc6e7bf463992b36c4279e007e7 = 'https://s3.eu-geo.objectstorage.softlayer.net'
else:
    endpoint_0041bdc6e7bf463992b36c4279e007e7 = 'https://s3.eu-geo.objectstorage.service.networklayer.com'

client_0041bdc6e7bf463992b36c4279e007e7 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='_WvAzxSMMbyyvT7Bo77QzhnGJ2DPckT5a9RkQmE9rCnN',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_0041bdc6e7bf463992b36c4279e007e7)

body = client_0041bdc6e7bf463992b36c4279e007e7.get_object(Bucket='courseracapstonedatascience-donotdelete-pr-mgnh2uarlhwkrt',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
#Merge the Coordinates with the Neighborhoods: 

neighborhoods_merged = neighborhoods_toronto
neighborhoods_merged = neighborhoods_merged.join(df_data_1.set_index('Postal Code'), on = 'PostalCode')
neighborhoods_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [26]:
neighborhoods_merged.to_csv('GeocoordinatesNeighborhoodsToronto.csv')
print('Daten gesichert')

Daten gesichert


# Part 3   Analyse, Clustering

In [16]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.12.0             |     pyhd8ed1ab_0          96 KB  conda-forge
    aiohttp-3.7.4              |   py37h5e8e339_0         632 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
   

py-xgboost-1.3.3     | 124 KB    | ##################################### | 100% 
tensorflow-base-1.14 | 87.6 MB   | ##################################### | 100% 
cytoolz-0.11.0       | 403 KB    | ##################################### | 100% 
testpath-0.4.4       | 85 KB     | ##################################### | 100% 
threadpoolctl-2.1.0  | 15 KB     | ##################################### | 100% 
absl-py-0.12.0       | 96 KB     | ##################################### | 100% 
libtiff-4.2.0        | 633 KB    | ##################################### | 100% 
cffi-1.14.5          | 225 KB    | ##################################### | 100% 
pysocks-1.7.1        | 27 KB     | ##################################### | 100% 
snappy-1.1.8         | 32 KB     | ##################################### | 100% 
python_abi-3.7       | 4 KB      | ##################################### | 100% 
dill-0.3.3           | 60 KB     | ##################################### | 100% 
biopython-1.78       | 2.6 M

mysql-common-8.0.23  | 1.5 MB    | ##################################### | 100% 
certifi-2020.12.5    | 143 KB    | ##################################### | 100% 
pcre-8.44            | 261 KB    | ##################################### | 100% 
json5-0.9.5          | 20 KB     | ##################################### | 100% 
python-dateutil-2.8. | 220 KB    | ##################################### | 100% 
gmp-6.2.1            | 806 KB    | ##################################### | 100% 
typing-extensions-3. | 8 KB      | ##################################### | 100% 
libgcc-ng-9.3.0      | 7.8 MB    | ##################################### | 100% 
dask-core-2021.3.0   | 702 KB    | ##################################### | 100% 
libpq-13.1           | 2.7 MB    | ##################################### | 100% 
numpy-1.20.1         | 5.8 MB    | ##################################### | 100% 
libzopfli-1.0.3      | 164 KB    | ##################################### | 100% 
liblapack-3.9.0      | 11 KB

secretstorage-3.3.1  | 24 KB     | ##################################### | 100% 
pyqtchart-5.12       | 256 KB    | ##################################### | 100% 
lz4-c-1.9.3          | 179 KB    | ##################################### | 100% 
libxcb-1.13          | 395 KB    | ##################################### | 100% 
setuptools-49.6.0    | 947 KB    | ##################################### | 100% 
idna-2.10            | 52 KB     | ##################################### | 100% 
pexpect-4.8.0        | 47 KB     | ##################################### | 100% 
pluggy-0.13.1        | 29 KB     | ##################################### | 100% 
termcolor-1.1.0      | 6 KB      | ##################################### | 100% 
lzo-2.10             | 314 KB    | ##################################### | 100% 
blosc-1.21.0         | 841 KB    | ##################################### | 100% 
tensorboard-1.14.0   | 3.2 MB    | ##################################### | 100% 
mistune-0.8.4        | 54 KB

done
Libraries imported.


In [17]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1937
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-main==custom=1937
done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          98 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy   

In [24]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [18]:
# get the Latitude and Longitude from Toronto to create a map
address = 'Toronto,ON'
geolocator = Nominatim(user_agent = "to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The georeaphic coordinates of Toronto are {},{}.'.format(latitude,longitude))


The georeaphic coordinates of Toronto are 43.6534817,-79.3839347.


In [19]:
# reduce the Dataset to Toronto
toronto_data = neighborhoods_merged[neighborhoods_merged['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [20]:
toronto_data.shape

(39, 5)

In [23]:
#Lets see How Torontos Neighborhoods look like
map_toronto = folium.Map(location=[latitude,longitude],zoom_start = 12)

for lat,lng,borough,neighborhood in zip (toronto_data['Latitude'],toronto_data['Longitude'],toronto_data['Borough'],toronto_data['Neighborhood']):
    label = '{},{}'.format (neighborhood,borough)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [25]:
CLIENT_ID = 'WCSAEPVN5OVR2CIQMKXONC04JS505C5CWKJXFYUWLFISIRAD' # your Foursquare ID
CLIENT_SECRET = 'QDG0CIEBA40GVXY2PDJQHIMDCAGHVM3LAYWOZMTNZCGGDUZG' # your Foursquare Secret 
ACCESS_TOKEN = '0N0RLQRFPBTP3JHYLM1S31FXICUDT3UJQIU5O0EZIQXYZAOJ' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WCSAEPVN5OVR2CIQMKXONC04JS505C5CWKJXFYUWLFISIRAD
CLIENT_SECRET:QDG0CIEBA40GVXY2PDJQHIMDCAGHVM3LAYWOZMTNZCGGDUZG


In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
# type your answer here
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )
toronto_venues.head()

Regent Park , Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Dufferin , Dovercourt Village
The Danforth  East
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
The Danforth West , Riverdale
Toronto Dominion Centre , Design Exchange
Brockton , Parkdale Village , Exhibition Place
India Bazaar , The Beaches West
Commerce Court , Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park , The Junction South
North Toronto West
The Annex , North Midtown , Yorkville
Parkdale , Roncesvalles
Davisville
University of Toronto , Harbord
Runnymede , Swansea
Moore Park , Summerhill East
Kensington Market , Chinatown , Grange Park
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Rosedale
Enclave of M5E
St. Jame

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park , Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [28]:
print(toronto_venues.shape)


(1578, 7)


In [29]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,60,60,60,60,60,60
"Brockton , Parkdale Village , Exhibition Place",24,24,24,24,24,24
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",15,15,15,15,15,15
Central Bay Street,60,60,60,60,60,60
Christie,16,16,16,16,16,16
Church and Wellesley,78,78,78,78,78,78
"Commerce Court , Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33
Davisville North,8,8,8,8,8,8


In [30]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.
